# Introduction/Motivation
TODO: Write an introduction
- What problems does Linear regression solve?
- Real-world examples where it is used
- Why understanding it deeply is important

# Dataset Loading

TODO: Load dataset for practice
1. Synthetic dataset: play with number of samples, features and noise
2. Real dataset: bostom housing

In [ ]:
from data.load_data import load_synthetic_regression, load_boston_dataset

# TODO: Load synthetic dataset
X, y = load_synthetic_regression(n_samples=100, n_features=1, noise=10, random_state=42)

# TODO: Explore dataset
# - Check shapes
# - Print first few rows
# - Plot X vs y (if 1 feature)

# Data processing
- Split dataset into train/test
- Normalize or standardize features
- Visualize distributions

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model definition

TODO: Initialize LinearRegression model
- Why learning rate matters
- Why we iterate multiple times
- How gradient descent updates parameters


In [ ]:
from ml.linear_model import LinearRegression

model = LinearRegression(lr=0.01, n_iters=1000)

# Training the model

TODO: Fit the model
- Track loss per iteration
- Experiment: what happens if we increase/decrease lr or n_iters?

In [ ]:
model.fit(X_train, y_train)

# TODO: Plot loss history

# Predictions and evaluation
TODO: Make predictions on test set
- Compute evaluations metrics
- Scatter plot: true vs predicted

In [ ]:
from ml.evaluation import mean_squared_error, r2_score

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print("MSE:", mse)
print("R²:", r2)

# Insights
- How weights relate to the slope of the line
- How bias affects predictions
- How the learning rate influences convergence
- How noise affects prediction accuracy
- What would happen with multiple features?